# library

In [1]:
%matplotlib notebook
import colorsys
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import pickle
import random
import seaborn
import warnings
import traceback
import pymysql
import joblib
import pandas as pd
import time
import collections
import math
import csv
import codecs

from tqdm import tqdm
from influxdb import InfluxDBClient
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from collections import defaultdict, OrderedDict
from fastdtw import fastdtw, dtw
from scipy.stats import norm
from sklearn.mixture import GaussianMixture
from scipy import interpolate
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from pandas import Series, DataFrame
from multiprocessing import Pool

from sklearn.cluster import AffinityPropagation
from sklearn import metrics
from itertools import cycle

warnings.filterwarnings('ignore')

# MySQL

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.sql import func
from sqlalchemy.dialects.mysql import LONGTEXT, DATETIME, INTEGER, BOOLEAN, DOUBLE
from sqlalchemy import Column, Integer, String, \
    Float, ForeignKey, text, Index, Date, BigInteger
from contextlib import contextmanager

db_uri = 'mysql+pymysql' + '://' + 'user' + ':' + '123456' + '@' + \
                   '10.11.112.202' + ':' + '3306' + '/' + 'SHUINI'
#print(db_uri)
engine = create_engine(db_uri, echo=False, pool_recycle=100)
Base = declarative_base(engine)
Session = sessionmaker(bind=engine)


@contextmanager
def session_scope():
    session = Session()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()
   
        
class DB_JiangYin(Base):
    __tablename__ = 'DB_JIANGYIN'
     
    ID = Column(INTEGER(11), primary_key=True)
    TIMESTAMP = Column(String(128))
    L0050 = Column(DOUBLE)
    L0049 = Column(DOUBLE)
    L0069 = Column(DOUBLE)
    L0060 = Column(DOUBLE)
    L0077 = Column(DOUBLE)
    L0026 = Column(DOUBLE)
    L0033 = Column(DOUBLE)
    L0036 = Column(DOUBLE)
    L0195 = Column(DOUBLE)


In [6]:
def get_db_JiangYin(OFFSET=0, LIMIT=1000000):
    res = []
    with session_scope() as session:
        evs = session.query(DB_JiangYin).offset(OFFSET).limit(LIMIT).all()
        for ev in evs:
            res.append([ev.TIMESTAMP, ev.L0050, ev.L0049, ev.L0069, ev.L0060, ev.L0077, ev.L0033, ev.L0036, ev.L0195])
    return res

# JiangYin Data Analysis

In [7]:
def utc_stamp_to_utc(timeStamp):
    timeStamp = timeStamp/1000
    dateArray = datetime.utcfromtimestamp(timeStamp)
    return dateArray

In [8]:
db = get_db_JiangYin()

In [26]:
csvfile = open("warn.csv","w",newline = "")
writer = csv.writer(csvfile)
writer.writerow(['北京时间','TIMESTAMP', 'STATE', '磨机振动量','立磨进出口压差','立磨出口温度','选粉机转速','主排风机速度','热风阀开度','冷风阀开度','循环阀开度'])
flag=0
cur_state = 0
for i in range(len(db)-1):
    if flag == 1 and db[i][j] != db[i+1][j]:
        cur_state = i+1
        print('two motions within 20s !')
    if flag == 1 and utc_stamp_to_utc(int(db[i+1][0])) - utc_stamp_to_utc(int(db[cur_state][0])) >= timedelta(seconds=20):
        print(utc_stamp_to_utc(int(db[i+1][0])))
        print(utc_stamp_to_utc(int(db[cur_state][0])))
        flag = 0
        writer.writerow([utc_stamp_to_utc(int(db[i+1][0])) + timedelta(hours=8), db[i+1][0], '结束', db[i+1][1], db[i+1][2], db[i+1][3], db[i+1][4], db[i+1][5], db[i+1][6], db[i+1][7], db[i+1][8]])
    if flag == 0:
        for j in range(4, 9):
            if db[i][j] != db[i+1][j]:
                flag = 1
                cur_state = i+1
                print(i)
                writer.writerow([utc_stamp_to_utc(int(db[i][0])) + timedelta(hours=8), db[i][0], '开始', db[i][1], db[i][2], db[i][3], db[i][4], db[i][5], db[i][6], db[i][7], db[i][8]])
                break
csvfile.close()

170
2018-08-20 21:18:18.840000
2018-08-20 21:17:58.840000
289
2018-08-20 21:22:26.840000
2018-08-20 21:22:06.840000
478
2018-08-20 21:29:58.840000
2018-08-20 21:29:38.840000
1179
2018-08-20 21:53:20.840000
2018-08-20 21:53:00.840000
1685
2018-08-20 22:10:24.840000
2018-08-20 22:10:04.840000
1825
2018-08-20 22:15:10.840000
2018-08-20 22:14:50.840000
2296
2018-08-20 22:31:46.840000
2018-08-20 22:31:26.840000
2687
2018-08-20 22:45:40.840000
2018-08-20 22:45:20.840000
3101
2018-08-20 23:05:03.481000
2018-08-20 23:04:43.340000
3316
2018-08-20 23:13:26.840000
2018-08-20 23:13:06.840000
3391
2018-08-20 23:16:16.840000
2018-08-20 23:15:56.840000
3926
2018-08-20 23:37:24.840000
2018-08-20 23:37:04.840000
3945
2018-08-20 23:38:02.840000
2018-08-20 23:37:42.840000
4688
2018-08-21 00:02:48.840000
2018-08-21 00:02:28.840000
5375
2018-08-21 00:25:44.840000
2018-08-21 00:25:24.840000
6850
2018-08-21 01:15:28.840000
2018-08-21 01:15:08.840000
11401
2018-08-21 03:48:44.840000
2018-08-21 03:48:24.840000

two motions within 20s !
2018-08-22 15:48:07.556000
2018-08-22 15:47:47.556000
74326
2018-08-22 16:17:39.556000
2018-08-22 16:17:19.556000
75274
2018-08-22 16:49:21.556000
2018-08-22 16:49:01.556000
75297
2018-08-22 16:50:07.556000
2018-08-22 16:49:47.556000
75314
2018-08-22 16:50:41.556000
2018-08-22 16:50:21.556000
75326
2018-08-22 16:51:05.556000
2018-08-22 16:50:45.556000
75344
2018-08-22 16:51:41.556000
2018-08-22 16:51:21.556000
75359
2018-08-22 16:52:11.556000
2018-08-22 16:51:51.556000
75433
two motions within 20s !
2018-08-22 16:54:59.556000
2018-08-22 16:54:39.556000
77474
2018-08-22 18:07:09.556000
2018-08-22 18:06:49.556000
77661
2018-08-22 18:13:23.556000
2018-08-22 18:13:03.556000
77677
2018-08-22 18:13:55.556000
2018-08-22 18:13:35.556000
77731
2018-08-22 18:15:43.556000
2018-08-22 18:15:23.556000
77745
2018-08-22 18:16:13.556000
2018-08-22 18:15:51.634000
77760
2018-08-22 18:16:41.556000
2018-08-22 18:16:21.556000
77773
2018-08-22 18:17:07.556000
2018-08-22 18:16:47.556

2018-08-24 07:37:27.556000
2018-08-24 07:37:07.556000
140674
2018-08-24 07:37:51.556000
2018-08-24 07:37:31.556000
140734
2018-08-24 07:39:51.556000
2018-08-24 07:39:31.556000
140763
2018-08-24 07:40:49.556000
2018-08-24 07:40:29.556000
140790
2018-08-24 07:41:43.556000
2018-08-24 07:41:23.556000
140825
2018-08-24 07:42:53.556000
2018-08-24 07:42:33.556000
140860
2018-08-24 07:44:03.556000
2018-08-24 07:43:43.556000
140900
2018-08-24 07:45:23.556000
2018-08-24 07:45:03.556000
141035
2018-08-24 07:49:55.556000
2018-08-24 07:49:35.556000
141134
2018-08-24 07:53:13.556000
2018-08-24 07:52:53.556000
141171
2018-08-24 07:54:27.556000
2018-08-24 07:54:07.556000
141198
2018-08-24 07:55:21.556000
2018-08-24 07:55:01.556000
141223
2018-08-24 07:56:11.556000
2018-08-24 07:55:51.556000
141934
2018-08-24 08:20:03.556000
2018-08-24 08:19:43.556000
223559
two motions within 20s !
two motions within 20s !
2018-08-26 08:05:47.556000
2018-08-26 08:05:27.556000
223752
2018-08-26 08:11:59.556000
2018-08-

2018-08-29 19:53:19.206000
364878
two motions within 20s !
two motions within 20s !
2018-08-29 19:56:47.206000
2018-08-29 19:56:27.206000
365086
2018-08-29 20:03:33.206000
2018-08-29 20:03:13.206000
365130
2018-08-29 20:05:01.206000
2018-08-29 20:04:41.206000
365348
2018-08-29 20:12:17.206000
2018-08-29 20:11:57.206000
366210
2018-08-29 20:41:01.206000
2018-08-29 20:40:41.206000
366233
2018-08-29 20:41:47.206000
2018-08-29 20:41:27.206000
366489
2018-08-29 20:50:21.206000
2018-08-29 20:50:01.206000
366860
2018-08-29 21:02:43.206000
2018-08-29 21:02:23.206000
367117
2018-08-29 21:11:21.206000
2018-08-29 21:11:01.206000
368586
2018-08-29 22:00:19.206000
2018-08-29 21:59:59.206000
370100
2018-08-29 22:50:47.206000
2018-08-29 22:50:27.206000
373196
2018-08-30 00:34:21.206000
2018-08-30 00:34:01.206000
374277
2018-08-30 01:10:27.206000
2018-08-30 01:10:07.206000
374574
2018-08-30 01:20:21.206000
2018-08-30 01:20:01.206000
375976
2018-08-30 02:07:05.206000
2018-08-30 02:06:45.206000
378115
2

522767
2018-09-02 12:01:29.206000
2018-09-02 12:01:09.206000
522784
2018-09-02 12:02:03.206000
2018-09-02 12:01:43.206000
522824
2018-09-02 12:03:23.206000
2018-09-02 12:03:03.206000
522923
2018-09-02 12:06:41.206000
2018-09-02 12:06:21.206000
522952
2018-09-02 12:07:39.206000
2018-09-02 12:07:19.206000
522974
2018-09-02 12:08:23.206000
2018-09-02 12:08:03.206000
523006
2018-09-02 12:09:41.206000
2018-09-02 12:09:20.752000
523199
two motions within 20s !
2018-09-02 12:16:21.206000
2018-09-02 12:16:01.206000
523221
two motions within 20s !
2018-09-02 12:17:13.206000
2018-09-02 12:16:53.206000
523255
2018-09-02 12:18:01.206000
2018-09-02 12:17:41.206000
523267
2018-09-02 12:18:25.206000
2018-09-02 12:18:05.206000
523281
2018-09-02 12:18:53.206000
2018-09-02 12:18:33.206000
523295
2018-09-02 12:19:21.206000
2018-09-02 12:19:01.206000
523313
two motions within 20s !
2018-09-02 12:20:41.206000
2018-09-02 12:20:19.815000
523336
2018-09-02 12:21:09.221000
2018-09-02 12:20:49.206000
523348
201

573553
2018-09-03 16:51:23.206000
2018-09-03 16:51:03.206000
573838
2018-09-03 17:10:23.206000
2018-09-03 17:10:03.206000
574063
2018-09-03 17:17:57.206000
2018-09-03 17:17:37.206000
574178
2018-09-03 17:21:47.206000
2018-09-03 17:21:27.206000
574286
2018-09-03 17:25:23.206000
2018-09-03 17:25:03.206000
574706
2018-09-03 17:39:23.206000
2018-09-03 17:39:03.206000
574846
2018-09-03 17:44:03.206000
2018-09-03 17:43:43.206000
574884
2018-09-03 17:45:19.221000
2018-09-03 17:44:59.221000
575020
2018-09-03 17:49:51.206000
2018-09-03 17:49:31.206000
575318
2018-09-03 17:59:59.206000
2018-09-03 17:59:39.206000
575359
2018-09-03 18:01:21.206000
2018-09-03 18:01:01.206000
575373
2018-09-03 18:01:49.206000
2018-09-03 18:01:29.206000
575384
2018-09-03 18:02:11.206000
2018-09-03 18:01:51.206000
575453
2018-09-03 18:04:31.206000
2018-09-03 18:04:11.206000
575494
2018-09-03 18:06:15.206000
2018-09-03 18:05:54.237000
575508
2018-09-03 18:06:41.206000
2018-09-03 18:06:21.206000
575536
2018-09-03 18:07:

2018-09-05 18:06:54.931000
2018-09-05 18:06:34.931000
657528
2018-09-05 18:07:16.931000
2018-09-05 18:06:56.931000
657550
2018-09-05 18:08:06.931000
2018-09-05 18:07:46.931000
657588
2018-09-05 18:09:34.947000
2018-09-05 18:09:02.931000
657661
two motions within 20s !
2018-09-05 18:12:38.931000
2018-09-05 18:12:18.931000
657713
2018-09-05 18:14:06.931000
2018-09-05 18:13:46.931000
657764
2018-09-05 18:15:48.931000
2018-09-05 18:15:28.931000
659034
2018-09-05 18:58:32.931000
2018-09-05 18:58:12.931000
659491
2018-09-05 19:14:02.931000
2018-09-05 19:13:42.931000
660596
2018-09-05 19:51:50.931000
2018-09-05 19:51:29.478000
662791
2018-09-05 21:08:12.931000
2018-09-05 21:07:52.931000
666123
2018-09-05 23:00:04.931000
2018-09-05 22:59:44.931000
667480
2018-09-05 23:45:50.931000
2018-09-05 23:45:30.931000
668417
2018-09-06 00:18:58.931000
2018-09-06 00:18:38.931000
668945
2018-09-06 00:36:34.931000
2018-09-06 00:36:14.931000
668960
2018-09-06 00:37:04.931000
2018-09-06 00:36:44.931000
669032

745263
2018-09-07 21:26:58.306000
2018-09-07 21:26:38.306000
745445
2018-09-07 21:33:02.306000
2018-09-07 21:32:42.306000
745511
2018-09-07 21:35:14.306000
2018-09-07 21:34:54.306000
745624
2018-09-07 21:39:00.306000
2018-09-07 21:38:40.306000
745732
2018-09-07 21:42:36.306000
2018-09-07 21:42:16.306000
745820
2018-09-07 21:45:32.306000
2018-09-07 21:45:12.306000
746010
2018-09-07 21:51:52.306000
2018-09-07 21:51:32.306000
746089
2018-09-07 21:54:30.306000
2018-09-07 21:54:10.306000
746516
2018-09-07 22:08:50.306000
2018-09-07 22:08:30.306000
746811
2018-09-07 22:23:14.306000
2018-09-07 22:22:54.306000
748694
2018-09-07 23:28:34.306000
2018-09-07 23:28:14.306000
749149
2018-09-07 23:44:02.306000
2018-09-07 23:43:42.306000
749907
2018-09-08 00:11:54.306000
2018-09-08 00:11:34.306000
750208
2018-09-08 00:21:58.306000
2018-09-08 00:21:38.306000
750225
2018-09-08 00:22:32.306000
2018-09-08 00:22:12.306000
752334
2018-09-08 01:33:22.306000
2018-09-08 01:33:02.306000
752883
2018-09-08 01:51:

2018-09-09 13:13:06.306000
2018-09-09 13:12:46.306000
812467
2018-09-09 13:29:46.306000
2018-09-09 13:29:26.306000
812558
2018-09-09 13:32:48.306000
2018-09-09 13:32:28.306000
812650
2018-09-09 13:35:52.306000
2018-09-09 13:35:32.306000
812744
2018-09-09 13:39:00.306000
2018-09-09 13:38:40.306000
812829
2018-09-09 13:41:50.306000
2018-09-09 13:41:30.306000
813256
2018-09-09 13:56:04.337000
2018-09-09 13:55:44.306000
813289
2018-09-09 13:57:12.306000
2018-09-09 13:56:52.306000
813374
2018-09-09 14:00:04.306000
2018-09-09 13:59:44.306000
813533
2018-09-09 14:05:22.306000
2018-09-09 14:05:02.306000
813822
2018-09-09 14:15:12.306000
2018-09-09 14:14:52.306000
813865
2018-09-09 14:16:38.306000
2018-09-09 14:16:18.306000
814058
2018-09-09 14:23:04.306000
2018-09-09 14:22:44.306000
814103
two motions within 20s !
two motions within 20s !
2018-09-09 14:24:46.306000
2018-09-09 14:24:26.306000
814119
2018-09-09 14:25:06.306000
2018-09-09 14:24:46.306000
814661
2018-09-09 14:44:38.306000
2018-09-

two motions within 20s !
2018-09-10 23:27:44.306000
2018-09-10 23:27:24.306000
870692
2018-09-10 23:34:04.306000
2018-09-10 23:33:44.306000
870749
2018-09-10 23:35:58.306000
2018-09-10 23:35:38.306000
871299
2018-09-10 23:54:18.306000
2018-09-10 23:53:58.306000
871364
2018-09-10 23:58:46.306000
2018-09-10 23:58:26.306000
871405
2018-09-11 00:00:08.306000
2018-09-10 23:59:48.306000
871561
2018-09-11 00:05:20.306000
2018-09-11 00:05:00.306000
871709
2018-09-11 00:10:16.306000
2018-09-11 00:09:56.306000
871865
2018-09-11 00:15:28.306000
2018-09-11 00:15:08.306000
872398
2018-09-11 00:33:14.306000
2018-09-11 00:32:54.306000
872480
2018-09-11 00:35:58.306000
2018-09-11 00:35:38.306000
872659
2018-09-11 00:41:56.306000
2018-09-11 00:41:36.306000
872898
2018-09-11 00:49:54.306000
2018-09-11 00:49:34.306000
873161
2018-09-11 00:58:40.306000
2018-09-11 00:58:20.306000
873331
2018-09-11 01:04:20.306000
2018-09-11 01:04:00.306000
877088
2018-09-11 03:09:34.306000
2018-09-11 03:09:14.306000
878702